# 3.1 Image Captioning with RNNs
在本次作业中，请您实现一个RNN，并使用它们训练一个做Image Caption的模型。

In [ ]:
# Setup cell.
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from mml.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from mml.rnn_layers import *
from mml.captioning_solver import CaptioningSolver
from mml.classifiers.rnn import CaptioningRNN
from mml.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from mml.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# COCO 数据集

在本次作业中，我们将使用 [COCO 数据集](https://cocodataset.org/) 的 2014 年版本，这是图像描述生成的标准测试集。该数据集包含 80,000 张训练图像和 40,000 张验证图像，每张图像都有由亚马逊机械土耳其工人编写的 5 条描述注释。

## 图像特征

我们已经预处理了数据并为您提取了特征。对于所有图像，我们从 VGG-16 网络（在 ImageNet 上预训练）中的 fc7 层提取了特征，这些特征存储在文件 `train2014_vgg16_fc7.h5` 和 `val2014_vgg16_fc7.h5` 中。为了减少处理时间和内存需求，我们使用主成分分析 (PCA) 将特征的维度从 4096 降至 512，这些特征存储在文件 `train2014_vgg16_fc7_pca.h5` 和 `val2014_vgg16_fc7_pca.h5` 中。

由于原始图像大小接近 20GB，因此我们未将其包含在下载内容中。由于所有图像均来自 Flickr，我们将训练和验证图像的 URL 存储在文件 `train2014_urls.txt` 和 `val2014_urls.txt` 中。这使得您可以按需下载图像以进行可视化。

## 描述文本

直接处理字符串效率较低，因此我们将使用编码版本的描述文本。每个单词被分配一个整数 ID，从而可以通过整数序列表示一个描述文本。整数 ID 和单词之间的映射存储在文件 `coco2014_vocab.json` 中，您可以使用文件 `mml/coco_utils.py` 中的函数 `decode_captions` 将整数 ID 的 NumPy 数组转换回字符串。

## 特殊标记

我们在词汇表中添加了一些特殊标记，并已为您处理了所有与特殊标记相关的实现细节：

- 在每个描述文本的开头和结尾分别添加一个特殊的 `<START>` 和 `<END>` 标记。
- 罕见的单词会被替换为特殊的 `<UNK>` 标记（表示“未知”）。
- 为了适应不同长度的描述文本，我们在 `<END>` 标记后为较短的描述文本添加特殊的 `<NULL>` 标记作为填充，并且不会对 `<NULL>` 标记计算损失或梯度。

## 数据加载

您可以使用文件 `mml/coco_utils.py` 中的函数 `load_coco_data` 加载所有 COCO 数据（描述文本、特征、URL 和词汇表）。运行以下代码单元即可加载数据：

In [ ]:
# Load COCO data from disk into a dictionary.
# We'll work with dimensionality-reduced features for the remainder of this assignment,
# but you can also experiment with the original features on your own by changing the flag below.
data = load_coco_data(pca_features=True)

# Print out all the keys and values from the data dictionary.
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

## 检查数据

在处理数据之前，查看数据集中的示例总是一个好主意。

您可以使用文件 `mml/coco_utils.py` 中的函数 `sample_coco_minibatch`，从 `load_coco_data` 返回的数据结构中采样小批量数据。运行以下代码以采样一小批训练数据并显示图像及其描述文本。多次运行并观察结果有助于更好地了解数据集。


In [ ]:
# Sample a minibatch and show the images and captions.
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
batch_size = 3

captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)
for i, (caption, url) in enumerate(zip(captions, urls)):
    plt.imshow(image_from_url(url))
    plt.axis('off')
    caption_str = decode_captions(caption, data['idx_to_word'])
    plt.title(caption_str)
    plt.show()

# Recurrent Neural Network


文件 `mml/rnn_layers.py` 包含循环神经网络所需的不同层类型的实现，而文件 `mml/classifiers/rnn.py` 则使用这些层实现了一个图像描述生成模型。

我们将首先在 `mml/rnn_layers.py` 中实现不同类型的 RNN 层。

## 注意事项

- **长短期记忆网络（LSTM）：** LSTM 是经典 RNN 的常见变体。本作业的附加文件 `LSTM_Captioning.ipynb` 是一个可选的额外练习内容，您可以选择忽略与 LSTM 相关的内容（如 `mml/classifiers/rnn.py` 和 `mml/rnn_layers.py` 中的 LSTM 引用）。
  
### 下一步
在 `mml/rnn_layers.py` 中实现标准的 RNN 层，并在模型中使用这些层为图像生成描述。

# 单步前向传播

打开文件 `mml/rnn_layers.py`。该文件实现了循环神经网络中常用层的前向传播和反向传播。

## 任务

首先实现函数 `rnn_step_forward`，它负责实现 Vanilla RNN 在单个时间步的前向传播。该函数的签名如下：

In [ ]:
N, D, H = 3, 10, 4

x = np.linspace(-0.4, 0.7, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.2, 0.5, num=N*H).reshape(N, H)
Wx = np.linspace(-0.1, 0.9, num=D*H).reshape(D, H)
Wh = np.linspace(-0.3, 0.7, num=H*H).reshape(H, H)
b = np.linspace(-0.2, 0.4, num=H)

next_h, _ = rnn_step_forward(x, prev_h, Wx, Wh, b)
expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

print('next_h error: ', rel_error(expected_next_h, next_h))

# Vanilla RNN: Step Backward
In the file `mml/rnn_layers.py` implement the `rnn_step_backward` function. After doing so run the following to numerically gradient check your implementation. You should see errors on the order of `e-8` or less.

In [ ]:
from mml.rnn_layers import rnn_step_forward, rnn_step_backward
np.random.seed(231)
N, D, H = 4, 5, 6
x = np.random.randn(N, D)
h = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_step_forward(x, h, Wx, Wh, b)

dnext_h = np.random.randn(*out.shape)

fx = lambda x: rnn_step_forward(x, h, Wx, Wh, b)[0]
fh = lambda prev_h: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_step_forward(x, h, Wx, Wh, b)[0]
fb = lambda b: rnn_step_forward(x, h, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dnext_h)
dprev_h_num = eval_numerical_gradient_array(fh, h, dnext_h)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dnext_h)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dnext_h)
db_num = eval_numerical_gradient_array(fb, b, dnext_h)

dx, dprev_h, dWx, dWh, db = rnn_step_backward(dnext_h, cache)

print('dx error: ', rel_error(dx_num, dx))
print('dprev_h error: ', rel_error(dprev_h_num, dprev_h))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# 前向传播（整个序列）

现在您已经完成了 Vanilla RNN 在单个时间步的前向传播和反向传播。接下来，我们需要将这些单步操作组合起来，实现一个可以处理完整数据序列的 RNN。

## 任务

在文件 `mml/rnn_layers.py` 中实现函数 `rnn_forward`。该函数将使用之前定义的 `rnn_step_forward` 来处理整个输入序列。


In [ ]:
N, T, D, H = 2, 3, 4, 5

x = np.linspace(-0.1, 0.3, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.3, 0.1, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.4, num=D*H).reshape(D, H)
Wh = np.linspace(-0.4, 0.1, num=H*H).reshape(H, H)
b = np.linspace(-0.7, 0.1, num=H)

h, _ = rnn_forward(x, h0, Wx, Wh, b)
expected_h = np.asarray([
  [
    [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
    [-0.39525808, -0.22554661, -0.0409454,   0.14649412,  0.32397316],
    [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525],
  ],
  [
    [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
    [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
    [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]])
print('h error: ', rel_error(expected_h, h))

# 反向传播（整个序列）

接下来，您需要实现 Vanilla RNN 对整个序列的反向传播。在文件 `mml/rnn_layers.py` 中完成函数 `rnn_backward` 的实现。该函数将调用之前实现的 `rnn_step_backward` 来完成对整个输入序列的反向传播。


In [ ]:
np.random.seed(231)

N, D, T, H = 2, 3, 10, 5

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = rnn_backward(dout, cache)

fx = lambda x: rnn_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: rnn_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: rnn_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print('dx error: ', rel_error(dx_num, dx))
print('dh0 error: ', rel_error(dh0_num, dh0))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# Word Embedding: 前向传播

在深度学习系统中，我们通常用向量来表示单词。词汇表中的每个单词都会与一个向量相关联，这些向量会随着系统的其余部分一起被训练。

在文件 `mml/rnn_layers.py` 中，您需要实现函数 `word_embedding_forward`，将单词（以整数表示）转换为向量。


In [ ]:
N, T, V, D = 2, 4, 5, 3

x = np.asarray([[0, 3, 1, 2], [2, 1, 0, 3]])
W = np.linspace(0, 1, num=V*D).reshape(V, D)

out, _ = word_embedding_forward(x, W)
expected_out = np.asarray([
 [[ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.42857143,  0.5,         0.57142857]],
 [[ 0.42857143,  0.5,         0.57142857],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429]]])

print('out error: ', rel_error(expected_out, out))

# Word Embedding: 反向传播

接下来，您需要在文件 `mml/rnn_layers.py` 中实现函数 `word_embedding_backward`，以完成单词嵌入的反向传播。该函数将计算嵌入矩阵的梯度。


In [ ]:
np.random.seed(231)

N, T, V, D = 50, 3, 5, 6
x = np.random.randint(V, size=(N, T))
W = np.random.randn(V, D)

out, cache = word_embedding_forward(x, W)
dout = np.random.randn(*out.shape)
dW = word_embedding_backward(dout, cache)

f = lambda W: word_embedding_forward(x, W)[0]
dW_num = eval_numerical_gradient_array(f, W, dout)

print('dW error: ', rel_error(dW, dW_num))

# Temporal Affine Layer

在每个时间步，我们使用一个仿射函数将该时间步的 RNN 隐藏向量转换为词汇表中每个单词的得分。由于这与您在作业 2 中实现的仿射层非常相似，我们已经为您提供了函数 `temporal_affine_forward` 和 `temporal_affine_backward`，它们位于文件 `mml/rnn_layers.py` 中。

## 检查代码

运行以下代码对 `temporal_affine_forward` 和 `temporal_affine_backward` 的实现进行数值梯度检查。您应该看到误差在 \(10^{-9}\) 或更小范围内。


In [ ]:
np.random.seed(231)

# Gradient check for temporal affine layer
N, T, D, M = 2, 3, 4, 5
x = np.random.randn(N, T, D)
w = np.random.randn(D, M)
b = np.random.randn(M)

out, cache = temporal_affine_forward(x, w, b)

dout = np.random.randn(*out.shape)

fx = lambda x: temporal_affine_forward(x, w, b)[0]
fw = lambda w: temporal_affine_forward(x, w, b)[0]
fb = lambda b: temporal_affine_forward(x, w, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dw_num = eval_numerical_gradient_array(fw, w, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

dx, dw, db = temporal_affine_backward(dout, cache)

print('dx error: ', rel_error(dx_num, dx))
print('dw error: ', rel_error(dw_num, dw))
print('db error: ', rel_error(db_num, db))

# Temporal Softmax Loss

在 RNN 语言模型中，每个时间步都会生成词汇表中每个单词的得分。我们根据每个时间步的真实单词标签，使用 softmax 损失函数计算损失和梯度。损失在时间步上累加，并在小批量内取平均。

## 特殊处理: `<NULL>` 标记

由于我们在小批量中操作，不同的描述文本可能具有不同的长度。为了统一长度，我们在每个描述文本的末尾补充 `<NULL>` 标记。这些 `<NULL>` 标记不应该计入损失或梯度。

因此，除了得分和真实标签之外，损失函数还接受一个 `mask` 数组，指示哪些得分计入损失。

### 实现细节

函数 `temporal_softmax_loss` 已为您实现，可以在文件 `mml/rnn_layers.py` 中查看其代码。接下来我们对该函数进行检查。

### 检查代码

运行以下代码对 `temporal_softmax_loss` 进行合理性检查和数值梯度检查。您应该看到 `dx` 的误差在 \(10^{-7}\) 或更小范围内。


In [ ]:
# Sanity check for temporal softmax loss
from mml.rnn_layers import temporal_softmax_loss

N, T, V = 100, 1, 10

def check_loss(N, T, V, p):
    x = 0.001 * np.random.randn(N, T, V)
    y = np.random.randint(V, size=(N, T))
    mask = np.random.rand(N, T) <= p
    print(temporal_softmax_loss(x, y, mask)[0])
  
check_loss(100, 1, 10, 1.0)   # Should be about 2.3
check_loss(100, 10, 10, 1.0)  # Should be about 23
check_loss(5000, 10, 10, 0.1) # Should be within 2.2-2.4

# Gradient check for temporal softmax loss
N, T, V = 7, 8, 9

x = np.random.randn(N, T, V)
y = np.random.randint(V, size=(N, T))
mask = (np.random.rand(N, T) > 0.5)

loss, dx = temporal_softmax_loss(x, y, mask, verbose=False)

dx_num = eval_numerical_gradient(lambda x: temporal_softmax_loss(x, y, mask)[0], x, verbose=False)

print('dx error: ', rel_error(dx, dx_num))

# RNN for Image Captioning

现在您已经实现了所需的各个层，接下来可以将它们组合起来构建一个图像描述生成模型。打开文件 `mml/classifiers/rnn.py`，查看 `CaptioningRNN` 类。

在 `loss` 函数中实现模型的前向和反向传播。目前您只需要实现 `cell_type='rnn'`（Vanilla RNN）的情况；稍后您将实现 LSTM 的逻辑。

完成后，运行以下代码以使用一个小测试用例检查您的前向传播实现。您应该看到误差在 \(10^{-10}\) 或更小范围内。

In [ ]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    hidden_dim=H,
    cell_type='rnn',
    dtype=np.float64
)

# Set all model parameters to fixed values
for k, v in model.params.items():
    model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)

features = np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D)
captions = (np.arange(N * T) % V).reshape(N, T)

loss, grads = model.loss(features, captions)
expected_loss = 9.83235591003

print('loss: ', loss)
print('expected loss: ', expected_loss)
print('difference: ', abs(loss - expected_loss))

运行以下代码单元以对 CaptioningRNN 类执行数值梯度检查；你应该看到误差在 e-6 量级或更小。

In [ ]:
np.random.seed(231)

batch_size = 2
timesteps = 3
input_dim = 4
wordvec_dim = 5
hidden_dim = 6
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
vocab_size = len(word_to_idx)

captions = np.random.randint(vocab_size, size=(batch_size, timesteps))
features = np.random.randn(batch_size, input_dim)

model = CaptioningRNN(
    word_to_idx,
    input_dim=input_dim,
    wordvec_dim=wordvec_dim,
    hidden_dim=hidden_dim,
    cell_type='rnn',
    dtype=np.float64,
)

loss, grads = model.loss(features, captions)

for param_name in sorted(grads):
    f = lambda _: model.loss(features, captions)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print('%s relative error: %e' % (param_name, e))

# 在小数据集上过拟合 RNN 描述生成模型

在本次作业中，我们使用 `CaptioningSolver` 类来训练图像描述生成模型。打开文件 `mml/captioning_solver.py`，阅读 `CaptioningSolver` 类

熟悉该 API 后，运行以下代码以确保你的模型能在 100 个训练样本的小数据集上过拟合。你应该看到最终的损失小于 0.1。

In [ ]:
np.random.seed(231)

small_data = load_coco_data(max_train=50)

small_rnn_model = CaptioningRNN(
    cell_type='rnn',
    word_to_idx=data['word_to_idx'],
    input_dim=data['train_features'].shape[1],
    hidden_dim=512,
    wordvec_dim=256,
)

small_rnn_solver = CaptioningSolver(
    small_rnn_model, small_data,
    update_rule='adam',
    num_epochs=50,
    batch_size=25,
    optim_config={
     'learning_rate': 5e-3,
    },
    lr_decay=0.95,
    verbose=True, print_every=10,
)

small_rnn_solver.train()

# Plot the training losses.
plt.plot(small_rnn_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

打印最终的训练损失。你应该看到最终的损失小于 0.1。

In [ ]:
print('Final loss: ', small_rnn_solver.loss_history[-1])

# 测试时的 RNN 采样

与分类模型不同，图像描述生成模型在训练时和测试时的行为差异很大。在训练时，我们可以使用真实的描述文本（ground-truth caption），因此在每个时间步将真实的单词作为输入提供给 RNN。在测试时，我们从词汇表的分布中采样每个时间步的输出，并将该采样结果作为下一时间步的 RNN 输入。

在文件 `mml/classifiers/rnn.py` 中实现测试时的 `sample` 方法。完成后，运行以下代码以在训练数据和验证数据上从你的过拟合模型中采样。训练数据上的采样结果应该非常好。然而，验证数据上的采样结果可能不会很合理。

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(small_data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = small_rnn_model.sample(features)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)          
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()